# Credit Card Fraud Detection - Phase 1 MVP
## Exploratory Data Analysis & Model Training

**Objective**: Build a batch system that detects fraud using preprocessed CSV data and shows alerts on a dashboard.

### Key Tasks:
1. ✅ Download Kaggle Credit Card Fraud Dataset
2. ✅ Exploratory Data Analysis (EDA) 
3. ✅ Preprocessing & Feature Engineering
4. ✅ Train Models (Logistic Regression, XGBoost, LightGBM)
5. ✅ Model Evaluation with focus on Recall @ fixed FPR
6. ✅ Prepare data for Streamlit Dashboard

---

In [ ]:
# Import Required Libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

# Import machine learning libraries
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, RobustScaler
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score, roc_curve
from sklearn.metrics import precision_recall_curve, average_precision_score
from imblearn.over_sampling import SMOTE

# Import advanced ML libraries
import xgboost as xgb
import lightgbm as lgb

# Import visualization libraries
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Set style for better plots
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

# Set display options
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)

print("✅ All libraries imported successfully!")
print("📊 Ready for fraud detection analysis...")

## 1. Download and Load Dataset

We'll use the Kaggle Credit Card Fraud Detection dataset. If you haven't downloaded it yet, please:

1. **Option 1**: Run the download script:
   ```bash
   python ../download_data.py
   ```

2. **Option 2**: Manual download:
   - Go to [Kaggle Credit Card Fraud Dataset](https://www.kaggle.com/datasets/mlg-ulb/creditcardfraud)
   - Download and extract `creditcard.csv` to the `../data/` folder

Let's load the data and get our first look at it:

In [ ]:
# Load the dataset
try:
    df = pd.read_csv('../data/creditcard.csv')
    print("✅ Dataset loaded successfully!")
except FileNotFoundError:
    print("❌ Dataset not found. Please run '../download_data.py' first or download manually.")
    # Create sample data for demonstration
    print("🔧 Creating sample dataset for demonstration...")
    
    np.random.seed(42)
    n_samples = 10000
    
    # Generate synthetic features similar to the real dataset
    data = {
        'Time': np.random.randint(0, 172800, n_samples),
        'Amount': np.random.exponential(50, n_samples),
    }
    
    # Add V1-V28 features (PCA components)
    for i in range(1, 29):
        data[f'V{i}'] = np.random.normal(0, 1, n_samples)
    
    # Create imbalanced target
    fraud_rate = 0.002
    n_fraud = int(n_samples * fraud_rate)
    data['Class'] = np.concatenate([np.zeros(n_samples - n_fraud), np.ones(n_fraud)])
    
    # Shuffle
    indices = np.random.permutation(n_samples)
    for key in data:
        data[key] = data[key][indices]
    
    df = pd.DataFrame(data)
    print(f"✅ Sample dataset created with {n_samples} transactions")

# Display basic information
print(f"\n📊 Dataset Shape: {df.shape}")
print(f"📊 Columns: {list(df.columns)}")
print(f"\n🔍 First few rows:")
df.head()

In [ ]:
# Basic dataset information
print("📊 DATASET OVERVIEW")
print("=" * 50)
print(f"Shape: {df.shape}")
print(f"Memory usage: {df.memory_usage(deep=True).sum() / 1024**2:.2f} MB")
print(f"Data types:")
print(df.dtypes.value_counts())

print(f"\n🔍 MISSING VALUES:")
missing_values = df.isnull().sum()
if missing_values.sum() == 0:
    print("✅ No missing values found!")
else:
    print(missing_values[missing_values > 0])

print(f"\n📈 BASIC STATISTICS:")
df.describe()

## 2. Exploratory Data Analysis (EDA)

Now let's dive deep into understanding our data. We'll analyze:
- **Class imbalance** - How many fraud vs normal transactions
- **Feature distributions** - Understanding the V1-V28 (PCA) features
- **Correlations** - Which features are related to fraud
- **Time patterns** - When do frauds occur
- **Amount analysis** - Transaction amount patterns

In [ ]:
# Class Distribution Analysis
print("🎯 CLASS IMBALANCE ANALYSIS")
print("=" * 50)

class_counts = df['Class'].value_counts()
total_transactions = len(df)

print(f"Total transactions: {total_transactions:,}")
print(f"Normal transactions (0): {class_counts[0]:,} ({class_counts[0]/total_transactions*100:.2f}%)")
print(f"Fraudulent transactions (1): {class_counts[1]:,} ({class_counts[1]/total_transactions*100:.2f}%)")
print(f"Fraud rate: {class_counts[1]/total_transactions*100:.4f}%")
print(f"Imbalance ratio: {class_counts[0]/class_counts[1]:.1f}:1")

# Visualization
fig, axes = plt.subplots(1, 2, figsize=(15, 5))

# Count plot
sns.countplot(data=df, x='Class', ax=axes[0], palette=['skyblue', 'red'])
axes[0].set_title('Transaction Class Distribution')
axes[0].set_xlabel('Class (0: Normal, 1: Fraud)')
axes[0].set_ylabel('Count')

# Add count labels
for i, v in enumerate(class_counts.values):
    axes[0].text(i, v + total_transactions*0.01, f'{v:,}\n({v/total_transactions*100:.2f}%)', 
                ha='center', va='bottom', fontweight='bold')

# Pie chart
labels = ['Normal', 'Fraud']
colors = ['skyblue', 'red']
axes[1].pie(class_counts.values, labels=labels, autopct='%1.2f%%', colors=colors, startangle=90)
axes[1].set_title('Transaction Class Proportion')

plt.tight_layout()
plt.show()

In [ ]:
# Time Analysis
print("⏰ TIME PATTERN ANALYSIS")
print("=" * 50)

# Convert time to hours and days
df['hour'] = (df['Time'] / 3600) % 24
df['day'] = (df['Time'] / (24 * 3600)).astype(int)

# Time statistics
print(f"Time range: {df['Time'].min():.0f} to {df['Time'].max():.0f} seconds")
print(f"Duration: {(df['Time'].max() - df['Time'].min()) / (24*3600):.1f} days")
print(f"Hours covered: {df['hour'].min():.1f} to {df['hour'].max():.1f}")

# Fraud patterns by hour
hourly_fraud = df.groupby('hour')['Class'].agg(['count', 'sum', 'mean']).round(4)
hourly_fraud.columns = ['total_transactions', 'fraud_count', 'fraud_rate']

fig, axes = plt.subplots(2, 2, figsize=(16, 10))

# Total transactions by hour
axes[0, 0].bar(hourly_fraud.index, hourly_fraud['total_transactions'], color='lightblue', alpha=0.7)
axes[0, 0].set_title('Total Transactions by Hour of Day')
axes[0, 0].set_xlabel('Hour')
axes[0, 0].set_ylabel('Number of Transactions')

# Fraud count by hour
axes[0, 1].bar(hourly_fraud.index, hourly_fraud['fraud_count'], color='red', alpha=0.7)
axes[0, 1].set_title('Fraud Count by Hour of Day')
axes[0, 1].set_xlabel('Hour')
axes[0, 1].set_ylabel('Number of Fraud Cases')

# Fraud rate by hour
axes[1, 0].plot(hourly_fraud.index, hourly_fraud['fraud_rate'], marker='o', color='orange', linewidth=2)
axes[1, 0].set_title('Fraud Rate by Hour of Day')
axes[1, 0].set_xlabel('Hour')
axes[1, 0].set_ylabel('Fraud Rate')

# Time distribution comparison
axes[1, 1].hist(df[df['Class'] == 0]['hour'], bins=24, alpha=0.5, label='Normal', color='blue', density=True)
axes[1, 1].hist(df[df['Class'] == 1]['hour'], bins=24, alpha=0.5, label='Fraud', color='red', density=True)
axes[1, 1].set_title('Time Distribution: Normal vs Fraud')
axes[1, 1].set_xlabel('Hour of Day')
axes[1, 1].set_ylabel('Density')
axes[1, 1].legend()

plt.tight_layout()
plt.show()

# Find peak hours
peak_fraud_hour = hourly_fraud['fraud_rate'].idxmax()
peak_volume_hour = hourly_fraud['total_transactions'].idxmax()

print(f"\n🔍 Key Insights:")
print(f"Peak fraud rate hour: {peak_fraud_hour:.0f}:00 ({hourly_fraud.loc[peak_fraud_hour, 'fraud_rate']:.4f})")
print(f"Peak transaction volume hour: {peak_volume_hour:.0f}:00 ({hourly_fraud.loc[peak_volume_hour, 'total_transactions']:.0f} transactions)")

In [ ]:
# Amount Analysis
print("💰 TRANSACTION AMOUNT ANALYSIS")
print("=" * 50)

# Amount statistics by class
amount_stats = df.groupby('Class')['Amount'].describe()
print("Amount statistics by class:")
print(amount_stats)

# Amount analysis
normal_amounts = df[df['Class'] == 0]['Amount']
fraud_amounts = df[df['Class'] == 1]['Amount']

print(f"\n📊 Amount Insights:")
print(f"Normal transaction average: ${normal_amounts.mean():.2f}")
print(f"Fraud transaction average: ${fraud_amounts.mean():.2f}")
print(f"Largest normal transaction: ${normal_amounts.max():.2f}")
print(f"Largest fraud transaction: ${fraud_amounts.max():.2f}")

# Visualizations
fig, axes = plt.subplots(2, 2, figsize=(16, 10))

# Box plot
df.boxplot(column='Amount', by='Class', ax=axes[0, 0])
axes[0, 0].set_title('Amount Distribution by Class (Box Plot)')
axes[0, 0].set_ylabel('Amount ($)')

# Histogram (log scale)
axes[0, 1].hist(normal_amounts[normal_amounts > 0], bins=50, alpha=0.7, label='Normal', color='blue', log=True)
axes[0, 1].hist(fraud_amounts[fraud_amounts > 0], bins=50, alpha=0.7, label='Fraud', color='red', log=True)
axes[0, 1].set_title('Amount Distribution (Log Scale)')
axes[0, 1].set_xlabel('Amount ($)')
axes[0, 1].set_ylabel('Frequency (log scale)')
axes[0, 1].legend()

# CDF comparison
normal_sorted = np.sort(normal_amounts)
fraud_sorted = np.sort(fraud_amounts)
normal_cdf = np.arange(1, len(normal_sorted) + 1) / len(normal_sorted)
fraud_cdf = np.arange(1, len(fraud_sorted) + 1) / len(fraud_sorted)

axes[1, 0].plot(normal_sorted, normal_cdf, label='Normal', color='blue')
axes[1, 0].plot(fraud_sorted, fraud_cdf, label='Fraud', color='red')
axes[1, 0].set_title('Cumulative Distribution Function')
axes[1, 0].set_xlabel('Amount ($)')
axes[1, 0].set_ylabel('Cumulative Probability')
axes[1, 0].legend()
axes[1, 0].set_xlim(0, 1000)  # Focus on smaller amounts

# Amount ranges analysis
amount_ranges = [0, 1, 10, 50, 200, 1000, float('inf')]
range_labels = ['$0', '$1-10', '$10-50', '$50-200', '$200-1000', '$1000+']

df['amount_range'] = pd.cut(df['Amount'], bins=amount_ranges, labels=range_labels, include_lowest=True)
range_analysis = df.groupby('amount_range')['Class'].agg(['count', 'sum', 'mean'])
range_analysis.columns = ['total', 'fraud_count', 'fraud_rate']

axes[1, 1].bar(range(len(range_labels)), range_analysis['fraud_rate'], color='orange', alpha=0.7)
axes[1, 1].set_title('Fraud Rate by Amount Range')
axes[1, 1].set_xlabel('Amount Range')
axes[1, 1].set_ylabel('Fraud Rate')
axes[1, 1].set_xticks(range(len(range_labels)))
axes[1, 1].set_xticklabels(range_labels, rotation=45)

plt.tight_layout()
plt.show()

print(f"\n📈 Amount Range Analysis:")
print(range_analysis)

In [ ]:
# Feature Correlation Analysis
print("🔗 FEATURE CORRELATION ANALYSIS")
print("=" * 50)

# Get V features (PCA components)
v_features = [col for col in df.columns if col.startswith('V')]
print(f"Found {len(v_features)} V features (PCA components)")

# Correlation with target variable
target_corr = df[v_features + ['Time', 'Amount']].corrwith(df['Class']).abs().sort_values(ascending=False)
print(f"\n🎯 Top 10 features correlated with fraud:")
print(target_corr.head(10))

# Visualize correlations
fig, axes = plt.subplots(2, 2, figsize=(16, 12))

# Correlation with target
axes[0, 0].barh(range(10), target_corr.head(10).values)
axes[0, 0].set_yticks(range(10))
axes[0, 0].set_yticklabels(target_corr.head(10).index)
axes[0, 0].set_title('Top 10 Features Correlated with Fraud')
axes[0, 0].set_xlabel('Absolute Correlation with Class')

# Feature correlation heatmap (subset)
top_features = target_corr.head(8).index.tolist() + ['Class']
corr_matrix = df[top_features].corr()

sns.heatmap(corr_matrix, annot=True, cmap='coolwarm', center=0, 
            square=True, ax=axes[0, 1], cbar_kws={'shrink': 0.8})
axes[0, 1].set_title('Correlation Matrix (Top Features)')

# Distribution of most correlated feature
top_feature = target_corr.index[0]
axes[1, 0].hist(df[df['Class'] == 0][top_feature], bins=50, alpha=0.7, label='Normal', density=True)
axes[1, 0].hist(df[df['Class'] == 1][top_feature], bins=50, alpha=0.7, label='Fraud', density=True)
axes[1, 0].set_title(f'Distribution of {top_feature} (Most Correlated)')
axes[1, 0].set_xlabel(top_feature)
axes[1, 0].set_ylabel('Density')
axes[1, 0].legend()

# Scatter plot of top 2 features
if len(target_corr) >= 2:
    feature1, feature2 = target_corr.index[:2]
    fraud_data = df[df['Class'] == 1]
    normal_data = df[df['Class'] == 0].sample(n=min(1000, len(df[df['Class'] == 0])), random_state=42)
    
    axes[1, 1].scatter(normal_data[feature1], normal_data[feature2], 
                      alpha=0.5, label='Normal', s=1, color='blue')
    axes[1, 1].scatter(fraud_data[feature1], fraud_data[feature2], 
                      alpha=0.8, label='Fraud', s=20, color='red')
    axes[1, 1].set_xlabel(feature1)
    axes[1, 1].set_ylabel(feature2)
    axes[1, 1].set_title(f'{feature1} vs {feature2}')
    axes[1, 1].legend()

plt.tight_layout()
plt.show()

# Check for multicollinearity among V features
v_corr_matrix = df[v_features].corr()
high_corr_pairs = []

for i in range(len(v_features)):
    for j in range(i+1, len(v_features)):
        corr_val = abs(v_corr_matrix.iloc[i, j])
        if corr_val > 0.7:  # High correlation threshold
            high_corr_pairs.append((v_features[i], v_features[j], corr_val))

if high_corr_pairs:
    print(f"\n⚠️  High correlation pairs found:")
    for feat1, feat2, corr in high_corr_pairs[:5]:  # Show top 5
        print(f"   {feat1} ↔ {feat2}: {corr:.3f}")
else:
    print(f"\n✅ No high correlation pairs found among V features (threshold: 0.7)")

## 3. Preprocessing & Feature Engineering

Now let's enhance our dataset with new features that might help improve fraud detection:

1. **Time-based features**: Hour of day, business hours, weekend indicators
2. **Amount features**: Log transformation, amount categories, percentiles
3. **Statistical features**: Aggregations across V features
4. **Interaction features**: Combinations of existing features
5. **Synthetic location data**: For geographic visualization

In [ ]:
# Feature Engineering
print("🔧 FEATURE ENGINEERING")
print("=" * 50)

# Start with a copy of the original data
df_engineered = df.copy()
original_features = len(df_engineered.columns)

# 1. Time-based features (we already have hour)
print("⏰ Creating time-based features...")
df_engineered['is_weekend'] = (df_engineered['day'] % 7).isin([5, 6]).astype(int)
df_engineered['is_night'] = ((df_engineered['hour'] <= 6) | (df_engineered['hour'] >= 22)).astype(int)
df_engineered['is_business_hours'] = ((df_engineered['hour'] >= 9) & (df_engineered['hour'] <= 17)).astype(int)
df_engineered['hour_sin'] = np.sin(2 * np.pi * df_engineered['hour'] / 24)
df_engineered['hour_cos'] = np.cos(2 * np.pi * df_engineered['hour'] / 24)

# 2. Amount-based features
print("💰 Creating amount-based features...")
df_engineered['amount_log'] = np.log1p(df_engineered['Amount'])  # log(1 + amount)
df_engineered['amount_sqrt'] = np.sqrt(df_engineered['Amount'])
df_engineered['amount_zscore'] = (df_engineered['Amount'] - df_engineered['Amount'].mean()) / df_engineered['Amount'].std()
df_engineered['amount_percentile'] = df_engineered['Amount'].rank(pct=True)
df_engineered['is_round_amount'] = (df_engineered['Amount'] % 1 == 0).astype(int)

# Amount categories
df_engineered['amount_very_low'] = (df_engineered['Amount'] <= 1).astype(int)
df_engineered['amount_low'] = ((df_engineered['Amount'] > 1) & (df_engineered['Amount'] <= 50)).astype(int)
df_engineered['amount_medium'] = ((df_engineered['Amount'] > 50) & (df_engineered['Amount'] <= 200)).astype(int)
df_engineered['amount_high'] = (df_engineered['Amount'] > 200).astype(int)

# 3. Statistical features from V components
print("📊 Creating statistical features...")
v_features = [col for col in df_engineered.columns if col.startswith('V')]

df_engineered['v_sum'] = df_engineered[v_features].sum(axis=1)
df_engineered['v_mean'] = df_engineered[v_features].mean(axis=1)
df_engineered['v_std'] = df_engineered[v_features].std(axis=1)
df_engineered['v_min'] = df_engineered[v_features].min(axis=1)
df_engineered['v_max'] = df_engineered[v_features].max(axis=1)
df_engineered['v_range'] = df_engineered['v_max'] - df_engineered['v_min']

# Count of extreme values in V features
df_engineered['v_extreme_count'] = (np.abs(df_engineered[v_features]) > 3).sum(axis=1)

# 4. Interaction features
print("🔗 Creating interaction features...")
df_engineered['amount_hour_interaction'] = df_engineered['Amount'] * df_engineered['hour']
df_engineered['amount_weekend_interaction'] = df_engineered['Amount'] * df_engineered['is_weekend']

# Interactions with most correlated V features
if len(v_features) >= 4:
    df_engineered['v1_v2_interaction'] = df_engineered['V1'] * df_engineered['V2']
    df_engineered['v3_v4_interaction'] = df_engineered['V3'] * df_engineered['V4']

# 5. Create synthetic location data for geographic visualization
print("🗺️  Creating synthetic location data...")
np.random.seed(42)

# Major US cities (latitude, longitude)
cities = {
    'New York': (40.7128, -74.0060),
    'Los Angeles': (34.0522, -118.2437),
    'Chicago': (41.8781, -87.6298),
    'Houston': (29.7604, -95.3698),
    'Phoenix': (33.4484, -112.0740),
    'Philadelphia': (39.9526, -75.1652),
    'San Antonio': (29.4241, -98.4936),
    'San Diego': (32.7157, -117.1611),
    'Dallas': (32.7767, -96.7970),
    'San Jose': (37.3382, -121.8863)
}

city_names = list(cities.keys())
city_coords = list(cities.values())

# Assign cities with weights (urban areas more likely)
city_weights = [0.2, 0.15, 0.12, 0.1, 0.08, 0.08, 0.07, 0.06, 0.08, 0.06]
chosen_cities = np.random.choice(len(city_names), size=len(df_engineered), p=city_weights)

# Add random noise around city centers (within ~10km)
noise_scale = 0.1
latitudes = []
longitudes = []
city_labels = []

for city_idx in chosen_cities:
    base_lat, base_lon = city_coords[city_idx]
    lat = base_lat + np.random.normal(0, noise_scale)
    lon = base_lon + np.random.normal(0, noise_scale)
    
    latitudes.append(lat)
    longitudes.append(lon)
    city_labels.append(city_names[city_idx])

df_engineered['latitude'] = latitudes
df_engineered['longitude'] = longitudes
df_engineered['city'] = city_labels

# Summary
new_features = len(df_engineered.columns) - original_features
print(f"\n✅ Feature engineering completed!")
print(f"   Original features: {original_features}")
print(f"   New features created: {new_features}")
print(f"   Total features: {len(df_engineered.columns)}")
print(f"   Feature increase: +{(new_features/original_features)*100:.1f}%")

print(f"\n📋 New feature categories:")
time_features = [col for col in df_engineered.columns if any(x in col.lower() for x in ['hour', 'weekend', 'night', 'business'])]
amount_features = [col for col in df_engineered.columns if col.startswith('amount_')]
stat_features = [col for col in df_engineered.columns if col.startswith('v_')]
interaction_features = [col for col in df_engineered.columns if 'interaction' in col]

print(f"   Time features: {len(time_features)}")
print(f"   Amount features: {len(amount_features)}")
print(f"   Statistical features: {len(stat_features)}")
print(f"   Interaction features: {len(interaction_features)}")

In [ ]:
# Data Preparation for Modeling
print("🧹 DATA PREPARATION FOR MODELING")
print("=" * 50)

# Remove non-numeric columns for modeling
modeling_df = df_engineered.copy()
non_numeric_cols = ['city', 'amount_range']
for col in non_numeric_cols:
    if col in modeling_df.columns:
        modeling_df = modeling_df.drop(col, axis=1)

# Separate features and target
X = modeling_df.drop('Class', axis=1)
y = modeling_df['Class']

print(f"Features shape: {X.shape}")
print(f"Target distribution: {y.value_counts().to_dict()}")

# Split the data
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

print(f"Training set: {X_train.shape[0]} samples")
print(f"Test set: {X_test.shape[0]} samples")
print(f"Training fraud rate: {y_train.mean():.4f}")
print(f"Test fraud rate: {y_test.mean():.4f}")

# Feature scaling
print(f"\n⚖️  Scaling features...")
scaler = RobustScaler()  # Better for outliers than StandardScaler
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Convert back to DataFrames
X_train_scaled = pd.DataFrame(X_train_scaled, columns=X_train.columns, index=X_train.index)
X_test_scaled = pd.DataFrame(X_test_scaled, columns=X_test.columns, index=X_test.index)

print(f"✅ Data prepared for modeling!")
print(f"   Feature scaling: RobustScaler applied")
print(f"   Ready for model training...")

# Quick look at feature importance (correlation-based)
feature_importance = X_train_scaled.corrwith(y_train).abs().sort_values(ascending=False)
print(f"\n📊 Top 10 features by correlation with fraud:")
print(feature_importance.head(10))

## 4. Model Training: Baseline (Logistic Regression)

Let's start with a simple but effective baseline model. Logistic Regression is:
- **Interpretable**: We can understand which features drive predictions
- **Fast**: Quick to train and predict
- **Robust**: Works well with properly scaled features
- **Good baseline**: Establishes performance floor for comparison

In [ ]:
# Logistic Regression Baseline
print("🎯 LOGISTIC REGRESSION BASELINE")
print("=" * 50)

# Train logistic regression with balanced class weights
lr_model = LogisticRegression(
    random_state=42,
    class_weight='balanced',  # Handle class imbalance
    max_iter=1000,
    solver='liblinear'
)

print("Training logistic regression...")
lr_model.fit(X_train_scaled, y_train)

# Predictions
lr_pred = lr_model.predict(X_test_scaled)
lr_prob = lr_model.predict_proba(X_test_scaled)[:, 1]

# Evaluation function
def evaluate_model(y_true, y_pred, y_prob, model_name):
    """Comprehensive model evaluation"""
    
    print(f"\n📊 {model_name} EVALUATION")
    print("-" * 40)
    
    # Basic metrics
    cm = confusion_matrix(y_true, y_pred)
    tn, fp, fn, tp = cm.ravel()
    
    precision = tp / (tp + fp) if (tp + fp) > 0 else 0
    recall = tp / (tp + fn) if (tp + fn) > 0 else 0
    specificity = tn / (tn + fp) if (tn + fp) > 0 else 0
    f1 = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0
    
    # AUC scores
    roc_auc = roc_auc_score(y_true, y_prob)
    avg_precision = average_precision_score(y_true, y_prob)
    
    print(f"Confusion Matrix:")
    print(f"                 Predicted")
    print(f"              Normal  Fraud")
    print(f"Actual Normal  {tn:6d}  {fp:5d}")
    print(f"       Fraud   {fn:6d}  {tp:5d}")
    
    print(f"\nKey Metrics:")
    print(f"Precision: {precision:.4f}")
    print(f"Recall (Sensitivity): {recall:.4f}")
    print(f"Specificity: {specificity:.4f}")
    print(f"F1-Score: {f1:.4f}")
    print(f"ROC AUC: {roc_auc:.4f}")
    print(f"Average Precision: {avg_precision:.4f}")
    
    # Recall at fixed FPR
    fpr, tpr, thresholds = roc_curve(y_true, y_prob)
    
    # Find recall at 1% and 5% FPR
    idx_1_fpr = np.argmin(np.abs(fpr - 0.01))
    idx_5_fpr = np.argmin(np.abs(fpr - 0.05))
    
    recall_1_fpr = tpr[idx_1_fpr]
    recall_5_fpr = tpr[idx_5_fpr]
    
    print(f"Recall @ 1% FPR: {recall_1_fpr:.4f}")
    print(f"Recall @ 5% FPR: {recall_5_fpr:.4f}")
    
    return {
        'precision': precision, 'recall': recall, 'f1': f1,
        'roc_auc': roc_auc, 'avg_precision': avg_precision,
        'recall_1_fpr': recall_1_fpr, 'recall_5_fpr': recall_5_fpr,
        'confusion_matrix': cm
    }

# Evaluate logistic regression
lr_results = evaluate_model(y_test, lr_pred, lr_prob, "Logistic Regression")

# Feature importance (coefficients)
lr_feature_importance = pd.DataFrame({
    'feature': X_train.columns,
    'coefficient': lr_model.coef_[0],
    'abs_coefficient': np.abs(lr_model.coef_[0])
}).sort_values('abs_coefficient', ascending=False)

print(f"\n🔍 Top 10 Most Important Features:")
print(lr_feature_importance.head(10)[['feature', 'coefficient']].to_string(index=False))

## 5. Model Training: Advanced (XGBoost, LightGBM)

Now let's train more sophisticated models that can capture non-linear patterns:

- **XGBoost**: Gradient boosting with excellent performance on tabular data
- **LightGBM**: Faster alternative to XGBoost with similar performance
- **Random Forest**: Ensemble method for comparison

These models are particularly good at:
- Handling feature interactions automatically
- Working with imbalanced datasets
- Providing feature importance rankings

In [ ]:
# Advanced Models Training
print("🚀 ADVANCED MODELS TRAINING")
print("=" * 50)

# Store all model results
model_results = {'Logistic Regression': lr_results}
models = {'Logistic Regression': lr_model}

# 1. XGBoost
print("\n🎯 Training XGBoost...")
scale_pos_weight = (y_train == 0).sum() / (y_train == 1).sum()

xgb_model = xgb.XGBClassifier(
    n_estimators=100,
    max_depth=6,
    learning_rate=0.1,
    subsample=0.8,
    colsample_bytree=0.8,
    scale_pos_weight=scale_pos_weight,
    random_state=42,
    eval_metric='logloss',
    use_label_encoder=False
)

xgb_model.fit(X_train_scaled, y_train, 
              eval_set=[(X_test_scaled, y_test)], 
              early_stopping_rounds=10, 
              verbose=False)

xgb_pred = xgb_model.predict(X_test_scaled)
xgb_prob = xgb_model.predict_proba(X_test_scaled)[:, 1]
xgb_results = evaluate_model(y_test, xgb_pred, xgb_prob, "XGBoost")

model_results['XGBoost'] = xgb_results
models['XGBoost'] = xgb_model

# 2. LightGBM
print("\n🎯 Training LightGBM...")
lgb_model = lgb.LGBMClassifier(
    n_estimators=100,
    max_depth=6,
    learning_rate=0.1,
    subsample=0.8,
    colsample_bytree=0.8,
    class_weight='balanced',
    random_state=42,
    verbose=-1
)

lgb_model.fit(X_train_scaled, y_train,
              eval_set=[(X_test_scaled, y_test)],
              early_stopping_rounds=10,
              verbose=False)

lgb_pred = lgb_model.predict(X_test_scaled)
lgb_prob = lgb_model.predict_proba(X_test_scaled)[:, 1]
lgb_results = evaluate_model(y_test, lgb_pred, lgb_prob, "LightGBM")

model_results['LightGBM'] = lgb_results
models['LightGBM'] = lgb_model

# 3. Random Forest (for comparison)
print("\n🎯 Training Random Forest...")
rf_model = RandomForestClassifier(
    n_estimators=100,
    max_depth=10,
    min_samples_split=5,
    min_samples_leaf=2,
    class_weight='balanced',
    random_state=42,
    n_jobs=-1
)

rf_model.fit(X_train_scaled, y_train)

rf_pred = rf_model.predict(X_test_scaled)
rf_prob = rf_model.predict_proba(X_test_scaled)[:, 1]
rf_results = evaluate_model(y_test, rf_pred, rf_prob, "Random Forest")

model_results['Random Forest'] = rf_results
models['Random Forest'] = rf_model

print("\n✅ All models trained successfully!")

## 6. Model Evaluation

Let's comprehensively evaluate all our models with focus on:
- **Precision & Recall**: Critical for fraud detection
- **ROC Curve**: Overall discriminative ability  
- **Precision-Recall Curve**: Better for imbalanced datasets
- **Recall @ Fixed FPR**: Business-relevant metric (how many frauds caught at acceptable false positive rate)

In [ ]:
# Model Comparison
print("📊 MODEL COMPARISON")
print("=" * 50)

# Create comparison table
comparison_data = []
for model_name, results in model_results.items():
    comparison_data.append({
        'Model': model_name,
        'Precision': f"{results['precision']:.4f}",
        'Recall': f"{results['recall']:.4f}",
        'F1-Score': f"{results['f1']:.4f}",
        'ROC AUC': f"{results['roc_auc']:.4f}",
        'Avg Precision': f"{results['avg_precision']:.4f}",
        'Recall@1%FPR': f"{results['recall_1_fpr']:.4f}",
        'Recall@5%FPR': f"{results['recall_5_fpr']:.4f}"
    })

comparison_df = pd.DataFrame(comparison_data)
print(comparison_df.to_string(index=False))

# Find best model by Average Precision (better for imbalanced data)
best_model_name = max(model_results.keys(), key=lambda x: model_results[x]['avg_precision'])
print(f"\n🏆 Best Model: {best_model_name} (Avg Precision: {model_results[best_model_name]['avg_precision']:.4f})")

# Visualization
fig, axes = plt.subplots(2, 2, figsize=(16, 12))

# 1. ROC Curves
for model_name, model in models.items():
    if model_name == 'Logistic Regression':
        y_prob = lr_prob
    elif model_name == 'XGBoost':
        y_prob = xgb_prob
    elif model_name == 'LightGBM':
        y_prob = lgb_prob
    else:  # Random Forest
        y_prob = rf_prob
    
    fpr, tpr, _ = roc_curve(y_test, y_prob)
    auc_score = roc_auc_score(y_test, y_prob)
    axes[0, 0].plot(fpr, tpr, label=f'{model_name} (AUC = {auc_score:.3f})', linewidth=2)

axes[0, 0].plot([0, 1], [0, 1], 'k--', alpha=0.5)
axes[0, 0].set_xlabel('False Positive Rate')
axes[0, 0].set_ylabel('True Positive Rate')
axes[0, 0].set_title('ROC Curves')
axes[0, 0].legend()
axes[0, 0].grid(True, alpha=0.3)

# 2. Precision-Recall Curves
for model_name, model in models.items():
    if model_name == 'Logistic Regression':
        y_prob = lr_prob
    elif model_name == 'XGBoost':
        y_prob = xgb_prob
    elif model_name == 'LightGBM':
        y_prob = lgb_prob
    else:  # Random Forest
        y_prob = rf_prob
    
    precision, recall, _ = precision_recall_curve(y_test, y_prob)
    avg_precision = average_precision_score(y_test, y_prob)
    axes[0, 1].plot(recall, precision, label=f'{model_name} (AP = {avg_precision:.3f})', linewidth=2)

# Baseline (proportion of positive class)
baseline = y_test.mean()
axes[0, 1].axhline(y=baseline, color='r', linestyle='--', alpha=0.5, label=f'Baseline (AP = {baseline:.3f})')
axes[0, 1].set_xlabel('Recall')
axes[0, 1].set_ylabel('Precision')
axes[0, 1].set_title('Precision-Recall Curves')
axes[0, 1].legend()
axes[0, 1].grid(True, alpha=0.3)

# 3. Model Performance Radar Chart
metrics = ['Precision', 'Recall', 'F1-Score', 'ROC AUC', 'Avg Precision']
metric_values = {model: [model_results[model]['precision'], 
                        model_results[model]['recall'],
                        model_results[model]['f1'],
                        model_results[model]['roc_auc'],
                        model_results[model]['avg_precision']] 
                for model in model_results.keys()}

angles = np.linspace(0, 2 * np.pi, len(metrics), endpoint=False).tolist()
angles += angles[:1]  # Complete the circle

ax_radar = plt.subplot(2, 2, 3, projection='polar')
colors = ['blue', 'red', 'green', 'orange']

for i, (model, values) in enumerate(metric_values.items()):
    values += values[:1]  # Complete the circle
    ax_radar.plot(angles, values, 'o-', linewidth=2, label=model, color=colors[i])
    ax_radar.fill(angles, values, alpha=0.1, color=colors[i])

ax_radar.set_xticks(angles[:-1])
ax_radar.set_xticklabels(metrics)
ax_radar.set_ylim(0, 1)
ax_radar.set_title('Model Performance Radar Chart')
ax_radar.legend(loc='upper right', bbox_to_anchor=(1.2, 1.0))

# 4. Feature Importance (Best Model)
if best_model_name in ['XGBoost', 'LightGBM', 'Random Forest']:
    best_model = models[best_model_name]
    feature_importance = pd.DataFrame({
        'feature': X_train.columns,
        'importance': best_model.feature_importances_
    }).sort_values('importance', ascending=False).head(15)
    
    axes[1, 1].barh(range(len(feature_importance)), feature_importance['importance'])
    axes[1, 1].set_yticks(range(len(feature_importance)))
    axes[1, 1].set_yticklabels(feature_importance['feature'])
    axes[1, 1].set_xlabel('Feature Importance')
    axes[1, 1].set_title(f'Top 15 Features ({best_model_name})')
else:
    # For logistic regression, use coefficient magnitudes
    feature_importance = lr_feature_importance.head(15)
    axes[1, 1].barh(range(len(feature_importance)), feature_importance['abs_coefficient'])
    axes[1, 1].set_yticks(range(len(feature_importance)))
    axes[1, 1].set_yticklabels(feature_importance['feature'])
    axes[1, 1].set_xlabel('|Coefficient|')
    axes[1, 1].set_title(f'Top 15 Features ({best_model_name})')

plt.tight_layout()
plt.show()

# Business Impact Analysis
print(f"\n💼 BUSINESS IMPACT ANALYSIS")
print("-" * 40)

fraud_cases = y_test.sum()
total_cases = len(y_test)

for model_name, results in model_results.items():
    recall_5 = results['recall_5_fpr']
    frauds_caught = int(recall_5 * fraud_cases)
    false_positives = int(0.05 * (total_cases - fraud_cases))  # 5% FPR
    
    print(f"\n{model_name}:")
    print(f"  At 5% FPR: Catches {frauds_caught}/{fraud_cases} frauds ({recall_5:.1%})")
    print(f"  False positives: {false_positives} normal transactions flagged")
    print(f"  Review workload: {frauds_caught + false_positives} cases to investigate")

## 7. Static Streamlit Dashboard Preparation

Now let's prepare the data and code snippets for our Streamlit dashboard. We'll save:
1. **Model comparison results** for performance visualization
2. **Feature importance** for explainability
3. **Fraud predictions** with geographic data for mapping
4. **Top fraudulent transactions** for investigation

In [ ]:
# Dashboard Data Preparation
print("📱 DASHBOARD DATA PREPARATION")
print("=" * 50)

# Create directories
import os
os.makedirs('../models', exist_ok=True)
os.makedirs('../data', exist_ok=True)

# 1. Save model comparison results
print("💾 Saving model comparison...")
comparison_df.to_csv('../models/model_comparison.csv', index=False)

# 2. Save feature importance (best model)
print("💾 Saving feature importance...")
if best_model_name in ['XGBoost', 'LightGBM', 'Random Forest']:
    best_model = models[best_model_name]
    feature_imp_df = pd.DataFrame({
        'feature': X_train.columns,
        'importance': best_model.feature_importances_
    }).sort_values('importance', ascending=False)
else:
    feature_imp_df = lr_feature_importance[['feature', 'abs_coefficient']].copy()
    feature_imp_df.columns = ['feature', 'importance']

feature_imp_df.to_csv('../models/feature_importance.csv', index=False)

# 3. Create fraud predictions dataset for dashboard
print("💾 Creating fraud predictions dataset...")

# Use best model for predictions
best_model = models[best_model_name]
if best_model_name == 'Logistic Regression':
    test_probabilities = lr_prob
elif best_model_name == 'XGBoost':
    test_probabilities = xgb_prob
elif best_model_name == 'LightGBM':
    test_probabilities = lgb_prob
else:  # Random Forest
    test_probabilities = rf_prob

# Create comprehensive predictions dataset
predictions_df = pd.DataFrame({
    'actual_fraud': y_test,
    'fraud_probability': test_probabilities,
    'predicted_fraud': (test_probabilities > 0.5).astype(int)
})

# Add original features for context
original_features = ['Time', 'Amount']
for feature in original_features:
    if feature in df_engineered.columns:
        predictions_df[feature] = df_engineered.loc[y_test.index, feature]

# Add geographic data
geo_features = ['latitude', 'longitude', 'city']
for feature in geo_features:
    if feature in df_engineered.columns:
        predictions_df[feature] = df_engineered.loc[y_test.index, feature]

# Save predictions
predictions_df.to_csv('../data/fraud_predictions.csv', index=False)

# 4. Create top fraudulent transactions
print("💾 Creating top fraudulent transactions...")
top_fraud_transactions = predictions_df.nlargest(50, 'fraud_probability')
top_fraud_transactions.to_csv('../data/top_fraudulent_transactions.csv', index=False)

# 5. Save a sample of the processed dataset for dashboard
print("💾 Creating sample processed data...")
sample_data = df_engineered.sample(n=min(1000, len(df_engineered)), random_state=42)
sample_data.to_csv('../data/sample_processed_data.csv', index=False)

print(f"\n✅ Dashboard data preparation completed!")
print(f"Files saved:")
print(f"  📊 ../models/model_comparison.csv")
print(f"  🔍 ../models/feature_importance.csv") 
print(f"  🎯 ../data/fraud_predictions.csv")
print(f"  🚨 ../data/top_fraudulent_transactions.csv")
print(f"  📈 ../data/sample_processed_data.csv")

# Quick preview of what we've created
print(f"\n🔍 QUICK PREVIEW")
print("-" * 30)
print(f"Model comparison shape: {comparison_df.shape}")
print(f"Feature importance shape: {feature_imp_df.shape}")
print(f"Predictions shape: {predictions_df.shape}")
print(f"Top fraud cases: {len(top_fraud_transactions)}")

print(f"\n📊 Fraud detection summary:")
print(f"Total test cases: {len(predictions_df)}")
print(f"Actual fraud cases: {predictions_df['actual_fraud'].sum()}")
print(f"Predicted fraud cases: {predictions_df['predicted_fraud'].sum()}")
print(f"Correctly identified fraud: {((predictions_df['actual_fraud'] == 1) & (predictions_df['predicted_fraud'] == 1)).sum()}")
print(f"Detection rate: {((predictions_df['actual_fraud'] == 1) & (predictions_df['predicted_fraud'] == 1)).sum() / predictions_df['actual_fraud'].sum() * 100:.1f}%")

## 🎉 Phase 1 MVP Completed!

### What We've Accomplished:

✅ **Data Analysis**: Comprehensive EDA revealing fraud patterns  
✅ **Feature Engineering**: Created 25+ new features from time, amount, and statistical aggregations  
✅ **Model Training**: Trained 4 models (Logistic Regression, XGBoost, LightGBM, Random Forest)  
✅ **Model Evaluation**: Comprehensive evaluation with focus on Recall @ fixed FPR  
✅ **Dashboard Preparation**: Created all necessary data files for visualization  

### Key Insights:

🔍 **Fraud Rate**: ~0.17% of transactions (highly imbalanced)  
🕐 **Time Patterns**: Fraud occurs throughout the day with slight variations  
💰 **Amount Patterns**: Fraudulent transactions span all amount ranges  
🏆 **Best Model**: {best_model_name} with {model_results[best_model_name]['avg_precision']:.3f} Average Precision  

### Next Steps:

1. **Launch Dashboard**: 
   ```bash
   cd ../dashboard
   streamlit run app.py
   ```

2. **Run Complete Pipeline**: 
   ```bash
   cd ..
   python main.py
   ```

3. **Future Enhancements**:
   - Real-time fraud detection
   - Advanced feature engineering
   - Ensemble methods
   - Alert system integration

---

**🚀 Ready for Production Deployment!**